# Azure AI Agent Service & Semantic Kernel ఉపయోగించి RAG

ఈ కోడ్ స్నిపెట్ `Azure AI Agent Service` మరియు `Semantic Kernel` ఉపయోగించి retrieval-augmented generation (RAG) కోసం Azure AI ఏజెంట్‌ను సృష్టించడం మరియు నిర్వహించడం ఎలా అనేది చూపిస్తుంది. ఏజెంట్ పొందిన కంటెక్స్ట్ ఆధారంగా యూజర్ ప్రశ్నలను ప్రాసెస్ చేసి, సరైన సమాధానాలను అందిస్తుంది.


## పరిసరాలను ప్రారంభించడం


SQLite వెర్షన్ ఫిక్స్  
మీరు ఈ లోపాన్ని ఎదుర్కొంటే:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
మీ నోట్‌బుక్ ప్రారంభంలో ఈ కోడ్ బ్లాక్‌ను అన్‌కామెంట్ చేయండి:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### ప్యాకేజీలను దిగుమతి చేసుకోవడం  
క్రింది కోడ్ అవసరమైన ప్యాకేజీలను దిగుమతి చేసుకుంటుంది:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# సెమాంటిక్ కర్నెల్ & Azure AI ఏజెంట్ సర్వీస్‌తో Retrieval-Augmented Generation

ఈ నమూనా **Azure AI ఏజెంట్ సర్వీస్** ఉపయోగించి **Retrieval-Augmented Generation (RAG)** ఎలా చేయాలో చూపిస్తుంది, ఇది భాషా మోడల్‌ను అప్లోడ్ చేసిన డాక్యుమెంట్ నుండి డొమైన్-స్పెసిఫిక్ కంటెక్స్ట్‌తో కలిపి పనిచేస్తుంది.

### ఇది ఎలా పనిచేస్తుంది

1. **డాక్యుమెంట్ అప్లోడ్**: ఒక markdown ఫైల్ (document.md) లోని సమాచారం (Contoso యొక్క ట్రావెల్ ఇన్సూరెన్స్ పాలసీ) ఏజెంట్ సర్వీస్‌కు అప్లోడ్ చేయబడుతుంది.

2. **వెక్టార్ స్టోర్ సృష్టి**: డాక్యుమెంట్‌ను వెక్టర్ స్టోర్‌లో సూచిక చేయడం ద్వారా దాని కంటెంట్‌పై సెమాంటిక్ సెర్చ్ చేయగలగడం సాధ్యమవుతుంది.

3. **ఏజెంట్ కాన్ఫిగరేషన్**: `gpt-4o` మోడల్ ఉపయోగించి ఒక ఏజెంట్‌ను కింది కఠినమైన సూచనలతో ప్రారంభించబడుతుంది:
   - డాక్యుమెంట్ నుండి పొందిన కంటెంట్ ఆధారంగా మాత్రమే ప్రశ్నలకు సమాధానం ఇవ్వాలి.
   - ప్రశ్న పరిధి బయట ఉంటే సమాధానం ఇవ్వడానికి నిరాకరించాలి.

4. **ఫైల్ సెర్చ్ టూల్ ఇంటిగ్రేషన్**: `FileSearchTool` ఏజెంట్‌తో రిజిస్టర్ చేయబడుతుంది, ఇది మోడల్‌ను సూచిక చేయబడిన డాక్యుమెంట్ నుండి సంబంధిత స్నిప్పెట్లు సెర్చ్ చేసి పొందడానికి అనుమతిస్తుంది.

5. **వినియోగదారుల పరస్పర చర్య**: వినియోగదారులు ప్రశ్నలు అడగవచ్చు. డాక్యుమెంట్‌లో సంబంధిత సమాచారం కనుగొనబడితే, ఏజెంట్ ఒక ఆధారపడిన సమాధానం ఉత్పత్తి చేస్తుంది.  
   లేకపోతే, డాక్యుమెంట్‌లో తగినంత సమాచారం లేదని ఏజెంట్ స్పష్టంగా స్పందిస్తుంది.


### ప్రధాన ఫంక్షన్


ముందుగా Azure CLI ఉపయోగించి `az login` ను నడపండి, తద్వారా `DefaultAzureCredential` ఉపయోగిస్తున్నప్పుడు సరైన authentication context అందించబడుతుంది. Azure AI Agent Service API కీలు ఉపయోగించదు.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

మీరు ఈ క్రింది విధంగా అవుట్‌పుట్‌ను చూడాలి:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దాని స్వదేశీ భాషలోని అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించాలి. కీలకమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
